In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from scipy.interpolate import CubicSpline
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [9, 9]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
GC_name = 'M15'
path = os.getcwd()
df = pd.read_csv("{}/{}.dat".format(path,GC_name),sep='\s+')

In [ ]:
plt.scatter(df['x'],df['y'],s=0.1);

In [ ]:
#determin center
x_guess,y_guess = 3000,3000
radius = 200
def centeroidnp(df):
    fit_stars = df[((df['x']-x_guess)**2 + (df['y']-y_guess)**2) < radius**2]
    length = len(fit_stars)
    x = fit_stars['x'].values
    y = fit_stars['y'].values
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    return sum_x/length, sum_y/length
x_cen, y_cen = centeroidnp(df)
print("X Center is: {}".format(x_cen))
print("Y Center is: {}".format(y_cen))

In [ ]:
#determine MSTO and SGB
plt.scatter(df['VIvega'],df['Vvega'],s=0.1,alpha=0.5,c='grey')
plt.xlim(0.4,0.8)
plt.ylim(17.5,20.5)
plt.scatter(0.478,19.4,s=0.7,c='r',label='MSTO')
plt.scatter(0.478+0.05,18.85,s=0.7,c='b',label='SGB')
plt.legend()
plt.gca().invert_yaxis()

$$\textup{V}_{\textup{MSTO}} = 19.4$$
$$\textup{VI}_{\textup{MSTO}} = 0.478$$
$$\textup{V}_{\textup{SGB}} = 18.85$$
$$\textup{VI}_{\textup{SGB}} = 0.483$$

In [ ]:
#plot the v vs i image within 3.5 mag from sgb
V_sgb = 18.85
V_min = V_sgb - 3.5
V_max = V_sgb + 3.5
df_first_cut = df[(df['Vvega'] < V_max) & (df['Vvega'] > V_min)]
plt.scatter(df_first_cut['Vvega'],df_first_cut['Ivega'],s=0.0001)
plt.xlim(V_min, V_max)
plt.ylim(14.4,21.9)
plt.savefig("{}/vi.png".format(path),dpi=500);